In [1]:
import pandas as pd
import numpy as np
import os
import json
import re
from itertools import permutations 
from matplotlib import pyplot as plt
import pickle
from google.colab import drive
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from scipy.sparse.linalg import svds
import networkx as nx
from sklearn import svm
from sklearn.decomposition import KernelPCA
from sklearn.utils import resample
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,precision_recall_curve, roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost
from sklearn import datasets
from sklearn.pipeline import Pipeline
from imblearn.combine import SMOTETomek

from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
folder = '/content/drive/My Drive/Hallo_Data/'
clean_folder = folder + 'clean_data/'
feat_folder = folder + 'features_added/'
message_folder = folder + 'clean_data/messages/'
m_folder = folder + 'messages_with_text/'
f_folder = '/content/drive/My Drive/Hallo_Data/winter_project/'

friends = pd.read_pickle(feat_folder + "friends_all_features.pkl")
users = pd.read_pickle(feat_folder + "users.pkl")
bans = pd.read_pickle(clean_folder + "bans.pkl")
message_features = pd.read_pickle(f_folder + 'message_features.pkl')

message_per_user_feats = message_features.groupby(['user_id']).mean()
message_per_user_feats['banned'] = message_per_user_feats.index.isin(bans.user_id.unique())
message_per_user_feats['banned'] = message_per_user_feats['banned'].apply(lambda x: int(x))
message_per_user_feats.replace([np.inf, -np.inf], np.nan, inplace=True)
message_per_user_feats.fillna(0, inplace=True)
message_per_user_feats = pd.merge(message_per_user_feats, users[['english_proficiency','friend_cnt', 'followed_cnt', 'suspended']].reset_index(), on='user_id', how='left')
message_per_user_feats['banned'] = message_per_user_feats['banned'] + message_per_user_feats['suspended']
message_per_user_feats['banned'] = message_per_user_feats['banned'].apply(lambda x: min(x,1))
message_per_user_feats.drop(columns=['suspended'], inplace=True)
message_per_user_feats.fillna(0, inplace=True)
message_per_user_feats.set_index('user_id', inplace=True)

In [0]:
def one_hot_encode(df, encoding_cols):
  for col in encoding_cols:
    # use pd.concat to join the new columns with your original dataframe
    df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    # now drop the original column 
    df.drop([col], axis=1, inplace=True)
  return df

In [0]:
def evaluate_model(true_y, pred_y, model, X_test, y_test):
    tn, fp, fn, tp = confusion_matrix(true_y, pred_y).ravel()
    print("TN:", tn, " FP:", fp, " FN:",fn, " TP:", tp)
    print("TPR: ", tp / (tp + fn))
    print("PPV: ", tp/(tp + fp)) #how many you correclty guessed were banned over how many you guessed were banned
    print("TNR: ", tn/(tn + fn))
    print("Accuracy:", model.score(X_test, y_test))


In [0]:

def downsample(df, col_name, majority_val):

  # Separate majority and minority classes
  df_majority = df[df[col_name] == majority_val]
  df_minority = df[df[col_name] != majority_val]
        # .sample(n=class_sizes, random_state=123)

  # Downsample majority class
  df_majority_downsampled = resample(df_majority, 
                                  # sample without replacement
                                  replace=False,    
                                  # to match minority class
                                  #n_samples=class_sizes, 
                                  n_samples=len(df_minority),
                                  random_state=42) 
  
  # Combine minority class with downsampled majority class
  df_downsampled = pd.concat([df_majority_downsampled, df_minority])
  return df_downsampled

In [0]:
# ###one hot encode data###
# ENCODING_COLS = ['native_country', 'native_lang']
# #add encoding col user features
# message_per_user_feats = message_per_user_feats.merge(users[ENCODING_COLS], left_on='user_id', right_on='user_id')
# #one hot encode these vals
# message_per_user_feats = one_hot_encode(message_per_user_feats, ENCODING_COLS)
# message_per_user_feats.head()

In [6]:
#define downsampled data
messages_ds = downsample(message_per_user_feats, 'banned', 0)
X_ds, y_ds = messages_ds.drop(columns=['banned', 'total_interactions']).values, messages_ds.banned.values
train_X_ds, test_X_ds, train_y_ds, test_y_ds = train_test_split(X_ds, y_ds, test_size=0.3, random_state=42)


#define down-upsampled data
smote_sampler = SMOTETomek()
X_test_dus, y_test_dus = test_X_ds, test_y_ds
X_train_dus, y_train_dus = smote_sampler.fit_resample(train_X_ds, train_y_ds)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
#Upsample Data
# smote_sampler = SMOTETomek()

# X, y = message_per_user_feats.drop(columns=['banned', 'total_interactions']).values, message_per_user_feats.banned.values
# X_us, y_ds = messages_ds.drop(columns=['banned', 'total_interactions']).values, messages_ds.banned.values
# upsampled_X, upsampled_y = smote_sampler.fit_resample(train_X, train_y)


# train_X, test_X, train_y, test_y = train_test_split(random_X, random_y, test_size=0.3, random_state=42)
# resampled_X, resampled_y = smote_sampler.fit_resample(train_X, train_y)

In [0]:
def find_best_model(steps, param_grid, train_X, train_y):
  """
    steps should be something like [('pca', PCA()), ('LDA', LinearDiscriminantAnalysis())]
  """
  pipe = Pipeline(steps=steps)

  search = GridSearchCV(pipe, 
                        param_grid, 
                        n_jobs=-1,
                        scoring="roc_auc")

  search.fit(train_X, train_y)

  best_estimator = search.best_estimator_
  best_parameters = search.best_params_

  #print best params
  print("Best Parameters", best_parameters)
  return best_estimator

In [0]:
from sklearn.random_projection import GaussianRandomProjection
# transformer = GaussianRandomProjection(random_state=rng)
# X_new = transformer.fit_transform(X)

def find_test_best_model(model_key, dim_reducer_key, X_train, y_train, X_test, y_test):
    #todo - define struct
    DIM_REDUCERS = {'nmf' : ['nmf', NMF(), {'nmf__n_components': [8, 10, 14, 16, 20]}],
                    'kpca' : ['kpca', KernelPCA(), {'kpca__kernel':['linear', 'sigmoid'],
                                                    'kpca__n_components':[10, 14, 16, 20]}],
                    'grp' :['grp', GaussianRandomProjection(), {'grp__n_components': [8, 10, 14, 16, 20]}],

    }



    MODELS = {'forest' : ['forest', RandomForestClassifier(verbose=True), {'forest__max_depth':[9, 15, 32, 64], \
                                                              'forest__n_estimators':[16, 32, 64, 100, 200], \
                                                              'forest__max_features':[6, 15, 30, 90]}], \
              
              'logreg' : ['logreg', LogisticRegression(), {'logreg__penalty' : ['l1', 'l2', 'elasticnet'], \
                                                          'logreg__C' : np.logspace(-4, 4, 20)}], \
              
              'lda' : ['lda', LinearDiscriminantAnalysis(), {'lda__solver': ['svd','lsqr','eigen']}],

              'svm' : ['svm', svm.SVC(), {'svm__C': [0.001, 0.01, 0.1, 1, 10],
                                          'svm__gamma': [0.001, 0.01, 0.1, 1],
                                          'svm__degree': [2,3,4]}],
              
              'gbc' : ['gbc', GradientBoostingClassifier(), {'gbc__n_estimators' : [50, 100, 150], #GBC's are roboust to over-fitting
                                                            'gbc__max_depth': [12, 25, 60],
                                                            'gbc__min_samples_split': [2, 5, 10],
                                                            'gbc__min_samples_leaf' : [2, 5, 10]}],
              
              'xgboost' : ['xgboost', xgboost.XGBClassifier(), {'xgboost__n_estimators': [64, 100],
                                                                'xgboost__reg_alpha' : np.linspace(0, 1, 3),
                                                                'xgboost__reg_lambda' : np.linspace(0, 1, 3),
                                                                'xgboost__gamma' : np.linspace(0, 1, 3)}],
                                                                #'xgboost__n_estimators': [64, 100, 150, 200],
                                                                #'xgboost__reg_alpha' : np.linspace(0, 1, 5),
                                                                #'xgboost__reg_lambda' : np.linspace(0, 1, 5),
                                                                #'xgboost__gamma' : np.linspace(0, 1, 5),
                                                                #'xgboost__learning_rate': np.linspace(0, 1, 5)}],

              'knn' : ['knn', KNeighborsClassifier(), {'knn__n_neighbors':[5, 10, 15, 20],
                                                      'knn__algorithm' : ['ball_tree', 'kd_tree'],
                                                      'knn__leaf_size' : [20, 30, 40]}]

    }
    dr_name, dr, dr_params = DIM_REDUCERS[dim_reducer_key]
    m_name, m, m_params = MODELS[model_key]

    print("Training & Testing {0} with {1}".format(m_name, m_params))

    param_grid = dr_params
    param_grid.update(m_params)
    steps = [(dr_name, dr), (m_name, m)]
    best_estimator = find_best_model(steps, param_grid, X_train, y_train)

    #test best model and print results
    evaluate_model(y_test, best_estimator.predict(X_test), best_estimator, X_test, y_test)

Down-Up-Sampling data testing

CODY RUNNABLES


In [0]:
find_test_best_model('svm', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

In [0]:
find_test_best_model('svm', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

In [0]:
find_test_best_model('forest', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

In [0]:
find_test_best_model('forest', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

ERIC RUNNABLES

In [0]:
find_test_best_model('logreg', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

In [0]:
find_test_best_model('logreg', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

In [18]:
find_test_best_model('xgboost', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

Training & Testing xgboost with {'xgboost__n_estimators': [64, 100], 'xgboost__reg_alpha': array([0. , 0.5, 1. ]), 'xgboost__reg_lambda': array([0. , 0.5, 1. ]), 'xgboost__gamma': array([0. , 0.5, 1. ])}
Best Parameters {'kpca__kernel': 'linear', 'kpca__n_components': 14, 'xgboost__gamma': 1.0, 'xgboost__n_estimators': 100, 'xgboost__reg_alpha': 0.0, 'xgboost__reg_lambda': 0.5}
TN: 370  FP: 76  FN: 93  TP: 324
TPR:  0.7769784172661871
PPV:  0.81
TNR:  0.7991360691144709
Accuracy: 0.8041714947856315


In [0]:
find_test_best_model('xgboost', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

KOHLER RUNNABLES

In [0]:
find_test_best_model('lda', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing lda with {'lda__solver': ['svd', 'lsqr', 'eigen']}
# Best Parameters {'kpca__kernel': 'poly', 'kpca__n_components': 10, 'lda__solver': 'svd'}
# TN: 441  FP: 5  FN: 319  TP: 98
# TPR:  0.23501199040767387
# PPV:  0.9514563106796117
# TNR:  0.5802631578947368
# Accuracy: 0.6245654692931634

In [0]:
find_test_best_model('lda', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing lda with {'lda__solver': ['svd', 'lsqr', 'eigen']}
# Best Parameters {'lda__solver': 'svd', 'nmf__n_components': 10}
# TN: 356  FP: 90  FN: 109  TP: 308
# TPR:  0.7386091127098321
# PPV:  0.7738693467336684
# TNR:  0.7655913978494624
# Accuracy: 0.7694090382387022

In [0]:
find_test_best_model('knn', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing knn with {'knn__n_neighbors': [5, 10, 15, 20], 'knn__algorithm': ['ball_tree', 'kd_tree'], 'knn__leaf_size': [20, 30, 40]}
# Best Parameters {'knn__algorithm': 'kd_tree', 'knn__leaf_size': 40, 'knn__n_neighbors': 20, 'kpca__kernel': 'linear', 'kpca__n_components': 8}
# TN: 369  FP: 77  FN: 103  TP: 314
# TPR:  0.7529976019184652
# PPV:  0.80306905370844
# TNR:  0.7817796610169492
# Accuracy: 0.7914252607184241

In [0]:
find_test_best_model('knn', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)


# Training & Testing knn with {'knn__n_neighbors': [5, 10, 15, 20], 'knn__algorithm': ['ball_tree', 'kd_tree'], 'knn__leaf_size': [20, 30, 40]}
# Best Parameters {'knn__algorithm': 'ball_tree', 'knn__leaf_size': 20, 'knn__n_neighbors': 15, 'nmf__n_components': 8}
# TN: 373  FP: 73  FN: 102  TP: 315
# TPR:  0.7553956834532374
# PPV:  0.8118556701030928
# TNR:  0.7852631578947369
# Accuracy: 0.7972190034762456

In [0]:
find_test_best_model('gbc', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

Training & Testing gbc with {'gbc__n_estimators': [50, 100, 150], 'gbc__max_depth': [12, 25, 60], 'gbc__min_samples_split': [2, 5, 10], 'gbc__min_samples_leaf': [2, 5, 10]}


In [0]:
find_test_best_model('gbc', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

In [0]:
# find_test_best_model('lda', 'pca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing lda with {'lda__solver': ['svd', 'lsqr', 'eigen']}
# Best Parameters {'lda__solver': 'svd', 'pca__n_components': 16}
# TN: 373  FP: 73  FN: 103  TP: 314
# TPR:  0.7529976019184652
# PPV:  0.8113695090439277
# TNR:  0.7836134453781513
# Accuracy: 0.7960602549246814


# find_test_best_model('lda', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing lda with {'lda__solver': ['svd', 'lsqr', 'eigen']}
# Best Parameters {'lda__solver': 'lsqr', 'nmf__n_components': 20}
# TN: 407  FP: 39  FN: 141  TP: 276
# TPR:  0.6618705035971223
# PPV:  0.8761904761904762
# TNR:  0.7427007299270073
# Accuracy: 0.7914252607184241

# find_test_best_model('lda', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing lda with {'lda__solver': ['svd', 'lsqr', 'eigen']}
# Best Parameters {'kpca__kernel': 'linear', 'kpca__n_components': 16, 'lda__solver': 'svd'}
# TN: 370  FP: 76  FN: 102  TP: 315
# TPR:  0.7553956834532374
# PPV:  0.8056265984654731
# TNR:  0.7838983050847458
# Accuracy: 0.7937427578215527

In [0]:
# find_test_best_model('svm', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing svm with {'svm__C': [0.001, 0.01, 0.1, 1, 10], 'svm__gamma': [0.001, 0.01, 0.1, 1], 'svm__degree': [2, 3, 4]}
# Best Parameters {'kpca__kernel': 'linear', 'kpca__n_components': 20, 'svm__C': 10, 'svm__degree': 2, 'svm__gamma': 0.001}
# TN: 382  FP: 64  FN: 101  TP: 316
# TPR:  0.7577937649880095
# PPV:  0.8315789473684211
# TNR:  0.7908902691511387
# Accuracy: 0.8088064889918888


# find_test_best_model('svm', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing svm with {'svm__C': [0.001, 0.01, 0.1, 1, 10], 'svm__gamma': [0.001, 0.01, 0.1, 1], 'svm__degree': [2, 3, 4]}
# Best Parameters {'nmf__n_components': 20, 'svm__C': 10, 'svm__degree': 2, 'svm__gamma': 0.1}
# TN: 406  FP: 40  FN: 125  TP: 292
# TPR:  0.7002398081534772
# PPV:  0.8795180722891566
# TNR:  0.7645951035781544
# Accuracy: 0.8088064889918888




# WITH GRID-SEARCHED PARAMETERS


In [15]:
# param_grid = {'C': [0.1, 1],
#               'gamma': [0.01, 0.1],
#               'degree': [2, 3]}


# clf = svm.SVC()
# svm_gs = GridSearchCV(clf, param_grid).fit(X_train_dus, y_train_dus)

# svm_best = svm_gs.best_estimator_
# pred_y = svm_best.predict(X_test_dus)
# evaluate_model(y_test_dus, pred_y, svm_best, X_test_dus, y_test_dus)
# print(svm_gs.best_params_)

# find_test_best_model('gbc', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# TN: 366  FP: 80  FN: 94  TP: 323
# TPR:  0.7745803357314148
# PPV:  0.8014888337468983
# TNR:  0.7956521739130434
# Accuracy: 0.7983777520278099
# {'C': 1, 'degree': 2, 'gamma': 0.01}


find_test_best_model('gbc', 'grp', X_train_dus, y_train_dus, X_test_dus, y_test_dus)


Training & Testing gbc with {'gbc__n_estimators': [50, 100, 150], 'gbc__max_depth': [12, 25, 60], 'gbc__min_samples_split': [2, 5, 10], 'gbc__min_samples_leaf': [2, 5, 10]}


KeyboardInterrupt: ignored

In [14]:
# find_test_best_model('knn', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing knn with {'knn__n_neighbors': [5, 10, 15, 20], 'knn__algorithm': ['ball_tree', 'kd_tree'], 'knn__leaf_size': [20, 30, 40]}
# Best Parameters {'knn__algorithm': 'kd_tree', 'knn__leaf_size': 30, 'knn__n_neighbors': 20, 'nmf__n_components': 8}
# TN: 379  FP: 67  FN: 107  TP: 310
# TPR:  0.7434052757793765
# PPV:  0.8222811671087533
# TNR:  0.779835390946502
# Accuracy: 0.7983777520278099


# find_test_best_model('knn', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)


# Training & Testing knn with {'knn__n_neighbors': [5, 10, 15, 20], 'knn__algorithm': ['ball_tree', 'kd_tree'], 'knn__leaf_size': [20, 30, 40]}
# Best Parameters {'knn__algorithm': 'ball_tree', 'knn__leaf_size': 20, 'knn__n_neighbors': 20, 'kpca__kernel': 'linear', 'kpca__n_components': 20}
# TN: 365  FP: 81  FN: 107  TP: 310
# TPR:  0.7434052757793765
# PPV:  0.7928388746803069
# TNR:  0.7733050847457628
# Accuracy: 0.7821552723059096

# find_test_best_model('knn', 'grp', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing knn with {'knn__n_neighbors': [5, 10, 15, 20], 'knn__algorithm': ['ball_tree', 'kd_tree'], 'knn__leaf_size': [20, 30, 40]}
# Best Parameters {'grp__n_components': 20, 'knn__algorithm': 'ball_tree', 'knn__leaf_size': 40, 'knn__n_neighbors': 20}
# TN: 357  FP: 89  FN: 99  TP: 318
# TPR:  0.762589928057554
# PPV:  0.7813267813267813
# TNR:  0.7828947368421053
# Accuracy: 0.7821552723059096

Training & Testing knn with {'knn__n_neighbors': [5, 10, 15, 20], 'knn__algorithm': ['ball_tree', 'kd_tree'], 'knn__leaf_size': [20, 30, 40]}
Best Parameters {'grp__n_components': 20, 'knn__algorithm': 'ball_tree', 'knn__leaf_size': 40, 'knn__n_neighbors': 20}
TN: 357  FP: 89  FN: 99  TP: 318
TPR:  0.762589928057554
PPV:  0.7813267813267813
TNR:  0.7828947368421053
Accuracy: 0.7821552723059096


/usr/local/lib/python3.6/dist-packages/sklearn/random_projection.py:390: DataDimensionalityWarning: The number of components is higher than the number of features: n_features < n_components (12 < 20).The dimensionality of the problem will not be reduced.
  DataDimensionalityWarning)


In [0]:
# find_test_best_model('gbc', 'nmf', X_train_dus, y_train_dus, X_test_dus, y_test_dus)


# Training & Testing gbc with {'gbc__n_estimators': [50, 100, 150], 'gbc__max_depth': [12, 25, 60], 'gbc__min_samples_split': [2, 5, 10], 'gbc__min_samples_leaf': [2, 5, 10]}
# Best Parameters {'gbc__max_depth': 12, 'gbc__min_samples_leaf': 10, 'gbc__min_samples_split': 10, 'gbc__n_estimators': 50, 'nmf__n_components': 10}
# TN: 364  FP: 82  FN: 92  TP: 325
# TPR:  0.7793764988009593
# PPV:  0.7985257985257985
# TNR:  0.7982456140350878
# Accuracy: 0.7983777520278099


# find_test_best_model('gbc', 'kpca', X_train_dus, y_train_dus, X_test_dus, y_test_dus)

# Training & Testing gbc with {'gbc__n_estimators': [50, 100, 150], 'gbc__max_depth': [12, 25, 60], 'gbc__min_samples_split': [2, 5, 10], 'gbc__min_samples_leaf': [2, 5, 10]}
# Best Parameters {'gbc__max_depth': 25, 'gbc__min_samples_leaf': 10, 'gbc__min_samples_split': 2, 'gbc__n_estimators': 50, 'kpca__kernel': 'linear', 'kpca__n_components': 20}
# TN: 366  FP: 80  FN: 93  TP: 324
# TPR:  0.7769784172661871
# PPV:  0.801980198019802
# TNR:  0.7973856209150327
# Accuracy: 0.7995365005793743